In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data.csv")

In [3]:
df.tail()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
169904,0.1730,"['DripReport', 'Tyga']",0.875,163800,0.443,1,4KppkflX7I3vJQk7urOJaS,0.000032,1,0.0891,-7.461,1,Skechers (feat. Tyga) - Remix,75,2020-05-15,0.1430,100.012,0.306,2020
169905,0.0167,"['Leon Bridges', 'Terrace Martin']",0.719,167468,0.385,0,1ehhGlTvjtHo2e4xJFB0SZ,0.031300,8,0.1110,-10.907,1,Sweeter (feat. Terrace Martin),64,2020-06-08,0.0403,128.000,0.270,2020
169906,0.5380,"['Kygo', 'Oh Wonder']",0.514,180700,0.539,0,52eycxprLhK3lPcRLbQiVk,0.002330,7,0.1080,-9.332,1,How Would I Know,70,2020-05-29,0.1050,123.700,0.153,2020
169907,0.0714,"['Cash Cash', 'Andy Grammer']",0.646,167308,0.761,0,3wYOGJYD31sLRmBgCvWxa4,0.000000,1,0.2220,-2.557,1,I Found You,70,2020-02-28,0.0385,129.916,0.472,2020
169908,0.1090,['Ingrid Andress'],0.512,214787,0.428,0,60RFlt48hm0l4Fu0JoccOl,0.000000,0,0.1050,-7.387,1,More Hearts Than Mine,65,2020-03-27,0.0271,80.588,0.366,2020


In [4]:
df_artista = df.artists

In [5]:
mask = (df['release_date'] > "1990-01-01") & (df['release_date'] <= "2020-05-15")

In [6]:
df_recorte = df.loc[mask]

In [7]:
df_recorte

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
5054,0.7750,['Paul Simon'],0.656,303507,0.6440,0,1RxlMG0bJhJdR5H6wL2oJf,0.416000,6,0.1200,-12.878,0,She Moves On,35,1990-10-16,0.0377,120.942,0.9430,1990
5055,0.1850,['Concrete Blonde'],0.275,308112,0.5980,0,30q9tmV1f3vHli0MhHtZR9,0.000011,3,0.1490,-6.757,1,"Tomorrow, Wendy",35,1990-09-16,0.0305,180.130,0.1980,1990
5057,0.9750,"['Roberta Flack', 'Donny Hathaway']",0.229,219600,0.0723,0,0vB3z9Jp1OFTV9gZDYkVG1,0.000176,9,0.0870,-17.460,1,For All We Know,35,1990-04-03,0.0402,79.303,0.0738,1990
5058,0.5590,['Benny Hinn'],0.120,239133,0.4190,0,2w3PjEG8DcqXADd51KwqtJ,0.000000,0,0.4860,-11.881,1,Holy Spirit Thou Art Welcome,37,1990-05-12,0.0427,73.289,0.2840,1990
5059,0.3960,['Elton John'],0.463,298027,0.6240,0,0EpRxgIqC79gXUMI8jPMVc,0.000013,3,0.0712,-9.174,1,You Gotta Love Someone,44,1990-11-08,0.0437,205.275,0.5470,1990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169902,0.0227,"['Trey Songz', 'Summer Walker']",0.619,194576,0.7190,1,5QZ11AHm7xiytOGXGlxQi5,0.000000,0,0.0839,-4.111,1,Back Home (feat. Summer Walker),69,2020-04-29,0.1570,86.036,0.3510,2020
169903,0.2100,"['LEGADO 7', 'Junior H']",0.795,218501,0.5850,0,52Cpyvd2dKb6XRn313nH87,0.000001,8,0.1120,-4.451,1,Ojos De Maniaco,68,2020-02-28,0.0374,97.479,0.9340,2020
169904,0.1730,"['DripReport', 'Tyga']",0.875,163800,0.4430,1,4KppkflX7I3vJQk7urOJaS,0.000032,1,0.0891,-7.461,1,Skechers (feat. Tyga) - Remix,75,2020-05-15,0.1430,100.012,0.3060,2020
169907,0.0714,"['Cash Cash', 'Andy Grammer']",0.646,167308,0.7610,0,3wYOGJYD31sLRmBgCvWxa4,0.000000,1,0.2220,-2.557,1,I Found You,70,2020-02-28,0.0385,129.916,0.4720,2020


In [8]:
artistas = df_recorte['artists'].tolist()

In [9]:
flat_list = []
lista_artistas_musicas = []
for sublist in artistas:
    sub = sublist.strip(" ] [ ").split(", ")
    lista_artistas_musicas.append(sub)
    for e in sub:
        flat_list.append(e.replace("'",""))

In [10]:
artistas_certo = []
sub_sub = []
for i in lista_artistas_musicas:
    sub_sub = []
    for e in i:
        e = e.replace("'","")
        sub_sub.append(e)
    artistas_certo.append(sub_sub)

In [11]:
df_recorte['artists'] = artistas_certo

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
total_artista = set(flat_list)


## criando dicionário com todos os artistas e sua popularidade média de músicas

In [13]:
pop = df_recorte["popularity"].tolist()

In [14]:
count = 0
pop_new = []
for a in artistas_certo:
    for g in a:
        pop_new.append(pop[count])
    count += 1

In [15]:
len(pop_new)

75543

In [16]:
len(flat_list)

75543

In [17]:
pop_musica_artista = pd.DataFrame({
    "artista":flat_list,
    "popularidade": pop_new
})

In [18]:
pop_musica_artista

,artista,popularidade
0,Paul Simon,35
1,Concrete Blonde,35
2,Roberta Flack,35
3,Donny Hathaway,35
4,Benny Hinn,37
...,...,...
75538,DripReport,75
75539,Tyga,75
75540,Cash Cash,70
75541,Andy Grammer,70


In [19]:
df_artista_media_pop = pop_musica_artista.groupby("artista").mean()

In [20]:
df_artista_media_pop

,popularidade
artista,
"""Adolescents Orquesta""",60.111111
"""Alexander ONeal""",45.333333
"""Amy Hanaialii""",37.000000
"""Aulii Cravalho""",66.800000
"""B. DAddario as Flounder""",44.000000
...,...
辛曉琪,46.500000
那英,46.500000
高慧君,52.000000
